Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple

Function to calculate IoU

In [3]:
@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

Enter the bounding box for the ground truth (box 1) followed by the bounding box of the predicted model(box 2)

### Image - lucas-hoang-rquNBlHgSWg-unsplash_jpg.rf.44d9d1748a02fdd36348b92739b7d130.jpg

In [18]:
box1 = np.array([0,0,401.39,337.78])
box2 = np.array([4,3,401.94,325.28])
calculate_iou(box1, box2, form='coco')

0.9432567932535302

Calculate the IoU by filling the function with the bounding boxes

In [19]:
box1 = np.array([0,41,216.67,299.7])
box2 = np.array([47,314,178.24,186.1])
calculate_iou(box1, box2, form='coco')

0.050153596988059185

In [20]:
box1 = np.array([1,414,214.3,359.64])
box2 = np.array([4,430,209.72,331.2])
calculate_iou(box1, box2, form='coco')

0.9015451088450485

In [21]:
box1 = np.array([170,293,242.46,289.13])
box2 = np.array([173,302,245.98,269.16])
calculate_iou(box1, box2, form='coco')

0.8973179626256211

In [22]:
box1 = np.array([375,327,183.57,254.28])
box2 = np.array([375,327,183.57,254.28])
calculate_iou(box1, box2, form='coco')

1.0

### Image - vladimir-fedotov-0f6d8tYEljU-unsplash_jpg.rf.778319bde7a7969eb32e3946c774235c.jpg

In [12]:
box1 = np.array([0,0,332.51,148])
box2 = np.array([1,1,331.84,176.89])
calculate_iou(box1, box2, form='coco')

0.8244452288770718

In [39]:
box1 = np.array([212,0,353.03,350.12])
box2 = np.array([213,1,362.64,348.25])
calculate_iou(box1, box2, form='coco')

0.9631533910133369

In [14]:
box1 = np.array([519,0,429.35,227.94])
box2 = np.array([554,0,379.92,258.62])
calculate_iou(box1, box2, form='coco')

0.7912809131951756

In [15]:
box1 = np.array([0,61,284.67,178.53])
box2 = np.array([1,57,340.13,181.15])
calculate_iou(box1, box2, form='coco')

0.8087163794828006

### Image - henley-design-studio-2ywzLqu-cuk-unsplash_jpg.rf.3a8b65abbadba355ad8e51c7b95790e9.jpg

In [6]:
box1 = np.array([509,122,455.88,488.49])
box2 = np.array([511,120,437,502])
calculate_iou(box1, box2, form='coco')

0.9339639150522172

### Image - audrey-martin-FJpHcqMud_Y-unsplash_jpg.rf.bba62c8515242f48efd6547a835ab5e7.jpg

In [31]:
box1 = np.array([219,29,211.79,229])
box2 = np.array([223,38,207.21,209.8])
calculate_iou(box1, box2, form='coco')

0.8967949213043278

In [32]:
box1 = np.array([109,157,174.83,167])
box2 = np.array([104,155,171.65,182.48])
calculate_iou(box1, box2, form='coco')

0.8521465317352966

In [33]:
box1 = np.array([287,213,156.17,160])
box2 = np.array([])
calculate_iou(box1, box2, form='coco')

0.0

In [34]:
box1 = np.array([178,251,186.81,189])
box2 = np.array([179,248,190.74,207.74])
calculate_iou(box1, box2, form='coco')

0.882629676678515

In [35]:
box1 = np.array([24,307,192.84,190])
box2 = np.array([22,310,195.33,185.15])
calculate_iou(box1, box2, form='coco')

0.9625566398720559

In [36]:
box1 = np.array([205,373,157.15,165])
box2 = np.array([])
calculate_iou(box1, box2, form='coco')

0.0

In [37]:
box1 = np.array([424,217,194.75,185])
box2 = np.array([433,221,190.79,180.81])
calculate_iou(box1, box2, form='coco')

0.9096388322310619

In [38]:
box1 = np.array([325,367,207.18,216])
box2 = np.array([318,372,209.08,201.02])
calculate_iou(box1, box2, form='coco')

0.8805951078790791

### Image - ayla-verschueren-UjOi6URY-C8-unsplash_jpg.rf.ec1015a78a0be01d13f34430c3caaa93.jpg

In [44]:
box1 = np.array([1,490,252.39,322.46])
box2 = np.array([0,490,257.71,314.37])
calculate_iou(box1, box2, form='coco')

0.9554313152064744

In [26]:
box1 = np.array([218,121,287.68,296.99])
box2 = np.array([0])
calculate_iou(box1, box2, form='coco')

0.9826693450691127

In [27]:
box1 = np.array([1199,411,300.08,307.77])
box2 = np.array([0])
calculate_iou(box1, box2, form='coco')

0.8913820936895052

In [28]:
box1 = np.array([1262,885,289.32,314.81])
box2 = np.array([0])
calculate_iou(box1, box2, form='coco')

0.8899651407307889

### Image - siora-photography-1VDWmab-Glk-unsplash_jpg.rf.38c8ae8104a036377c0b74344aa14731.jpg

In [40]:
box1 = np.array([2272,0,1184.36,1278.52])
box2 = np.array([2300,0,1142.1,1244.19])
calculate_iou(box1, box2, form='coco')

0.9384745559206509

In [41]:
box1 = np.array([871,0,1404.28,1244.77])
box2 = np.array([932,0,1313.91,1234.19])
calculate_iou(box1, box2, form='coco')

0.9277459381730289

In [42]:
box1 = np.array([1559,570,1509.58,1928.34])
box2 = np.array([1616,594,1460.98,1769.75])
calculate_iou(box1, box2, form='coco')

0.8786842145033076

In [43]:
box1 = np.array([1196,1215,1155.31,1544.36])
box2 = np.array([1177,1236,996.92,1387.99])
calculate_iou(box1, box2, form='coco')

0.7498514134430125

### Image - anne-nygard-1hxeCZ-S65M-unsplash_jpg.rf.fb6926255b5e4e205c0d140f8707364e.jpg

In [22]:
box1 = np.array([412,343,56.24,65.18])
box2 = np.array([411,345,57.89,73.94])
calculate_iou(box1, box2, form='coco')

0.8113757445673373

In [23]:
box1 = np.array([358,357,81.43,57.03])
box2 = np.array([358,357,62.11,65.94])
calculate_iou(box1, box2, form='coco')

0.6850847490601363

In [24]:
box1 = np.array([436,379,67.81,58.51])
box2 = np.array([442,379,58.75,57.65])
calculate_iou(box1, box2, form='coco')

0.8557845033476044

In [25]:
box1 = np.array([381,386,56.62,52.59])
box2 = np.array([382,390,54.96,55.36])
calculate_iou(box1, box2, form='coco')

0.8004881869094719

In [26]:
box1 = np.array([419,423,43.74,50.74])
box2 = np.array([416,425,47.03,48.06])
calculate_iou(box1, box2, form='coco')

0.886396756392214

In [27]:
box1 = np.array([368,435,49.59,43.7])
box2 = np.array([366,426,50.11,52.06])
calculate_iou(box1, box2, form='coco')

0.7701725192855768

### Image - siamak-cEVBsYy3gxU-unsplash_jpg.rf.e07e16caf07207f22110b42ecf2d4320.jpg

In [10]:
box1 = np.array([0,85,79.28,212.69])
box2 = np.array([0,80,78.55,219.24])
calculate_iou(box1, box2, form='coco')

0.9616970515140688

In [11]:
box1 = np.array([60,9,171.75,200.47])
box2 = np.array([60,1,178.31,196.85])
calculate_iou(box1, box2, form='coco')

0.8749527479689679

In [12]:
box1 = np.array([72,125,150.46,194.3])
box2 = np.array([55,129,168.31,182.9])
calculate_iou(box1, box2, form='coco')

0.8475703176784974

### Image - gift-habeshaw-kBhmgL0ngHw-unsplash_jpg.rf.4bcc3183f16d6a93d61822377b341c06.jpg

In [23]:
box1 = np.array([214,0,157.16,110.99])
box2 = np.array([215,4,157.25,100.06])
calculate_iou(box1, box2, form='coco')

0.8911541660909429

In [24]:
box1 = np.array([378,0,81.29,14.18])
box2 = np.array([])
calculate_iou(box1, box2, form='coco')

0.0

In [25]:
box1 = np.array([562,46,107.24,108.53])
box2 = np.array([561,44,108.05,112.78])
calculate_iou(box1, box2, form='coco')

0.9522225321813662

In [26]:
box1 = np.array([564,4,115.25,101.75] )
box2 = np.array([566,0,110.5,79.77])
calculate_iou(box1, box2, form='coco')

0.69082970577702

In [27]:
box1 = np.array([648,59,102.9,100.51] )
box2 = np.array([658,51,86.52,87.35])
calculate_iou(box1, box2, form='coco')

0.6252512992519194

In [28]:
box1 = np.array([1238,2,122.62,118.39])
box2 = np.array([1241,0,113.65,115.48])
calculate_iou(box1, box2, form='coco')

0.8756923187204043

In [29]:
box1 = np.array([1333,0,124.44,114.69] )
box2 = np.array([1337,0,126.35,114.05])
calculate_iou(box1, box2, form='coco')

0.9196669866378119

In [30]:
box1 = np.array([1313,100,97.94,86.33])
box2 = np.array([1310,96,100.92,77.08])
calculate_iou(box1, box2, form='coco')

0.7904764590230908

### Image - miops-trigger-sy0HX0_mzG0-unsplash_jpg.rf.dde7f7af38757ec63b094108aed277c0.jpg

In [41]:
box1 = np.array([429,125,453.47,329.51])
box2 = np.array([0])
calculate_iou(box1, box2, form='coco')

0.983601318573747

In [45]:
box1 = np.array([859,12,424.62,386.1])
box2 = np.array([865,10,420.8,402.8])
calculate_iou(box1, box2, form='coco')

0.9408583394962531